**Reference**: 
- [Advanced R: S4](https://adv-r.hadley.nz/s4.html)

In [1]:
library(methods)
library(sloop)

# 1. Introduction

# 2. Basic

Components:
- Class definition: `setClass()`
- Generics: `setGeneric()`
- Methods: `setMethod()`

## 2.1 Class

> Define a class with **`SetClass("class name", slots, prototype)`**

<b style = "color:red">If you want a slot to be any class, use pseudo-class "ANY"</b>

In [2]:
setClass(
    # class name
    "Person",
    # define slots and class of slots
    slots = c(name = "character", age = "numeric"),
    # define the default values of slots (not compulsary but you should always include)
    prototype = list(name = NA_character_, age = NA_real_)
)

> Create a new instance of a class with **`new("class name", ...)`** where `...` is the value for slot

In [3]:
musk <- new("Person", name = "Elon Musk")
musk

An object of class "Person"
Slot "name":
[1] "Elon Musk"

Slot "age":
[1] NA


In [4]:
# get class name of an instance
is(musk)

[1] "Person"

> See **`?slot`** to work with slot

In [5]:
# get value with @ in S4 (equivalent $ in S3)
musk@name
# get value with `slot` in S4 (equivalent to `[[` in S3)
slot(musk, "name")

[1] "Elon Musk"

[1] "Elon Musk"

In [6]:
# set value
musk@age <- 42
musk@age
# equivalent
slot(musk, "age") <- 42
musk@age

[1] 42

[1] 42

<b style = "color:red">NOTE: In practice, you should only use `@` for method. For getting and setting slots, use `getter` and `setter`</b>


In [7]:
# get the slot names of an instance
slotNames(musk)

[1] "name" "age"

In [8]:
# get the slot names and slots class for a class
getSlots("Person")

name         age 
"character"   "numeric"

In [9]:
# check if an instance has a slot 
.hasSlot(musk, "network")

[1] FALSE

## 2.2 Generics

>Create a new generic with 
>```r
>setGeneric("generic name", function(self, ...) standardGeneric("generic name"))
>```

In [10]:
# turn function age into a generic function
setGeneric("age", function(self) standaradGeneric("age"))
# turn fucntion `age<-` into a generic function
setGeneric("age<-", function(self, value) standaradGeneric("age<-"))

[1] "age"

[1] "age<-"

## 2.3 Methods

Given a generic, define methods for a specific class

> ```r
>setMethod("method name", "class to work", function(self, ...) code)
>```

In [11]:
# create getter that extract the value of slot age from class Person
# think of age.Person <- function(....) in S3
setMethod("age", "Person", function(self) self@age)
# setter for slot age from class Person
setMethod("age<-", "Person", function(self, value) {
    self@age <- value
    self
})

In [12]:
# getter
age(musk)

[1] 42

In [13]:
# setter
age(musk) <- 45
age(musk)

[1] 45

## 2.4 Inheritances

<b style = "color:red">Provide multiple inheritance (a class can inherit from multiple class, in S3, a class only can inherit from 1 class)</b>

> inherit with **`SetClass(contains = c("parent1", "parent2", ...))`**

In [14]:
setClass(
    # class name
    "Employee",
    # names of classes that this class inherits from
    contains = "Person",
    slots = c(boss = "Person"), 
    prototype = list(boss = new("Person"))
)

In [15]:
str(new("Employee"))

Formal class 'Employee' [package ".GlobalEnv"] with 3 slots
  ..@ boss:Formal class 'Person' [package ".GlobalEnv"] with 2 slots
  .. .. ..@ name: chr NA
  .. .. ..@ age : num NA
  ..@ name: chr NA
  ..@ age : num NA


## 2.5 Identify S4 class and its methods, generics

In [16]:
sloop::otype(musk)

[1] "S4"

In [17]:
sloop::ftype(age)

[1] "S4"      "generic"

In [18]:
# methods of S4 class "Person"
sloop::s4_methods_class("Person")

generic,class,visible,source
<chr>,<chr>,<lgl>,<chr>
age,Person,TRUE,R_GlobalEnv
age<-,Person,TRUE,R_GlobalEnv


In [19]:
# S4 classes having generic age
sloop::s4_methods_generic("age")

generic,class,visible,source
<chr>,<chr>,<lgl>,<chr>
age,ANY,TRUE,R_GlobalEnv
age,Person,TRUE,R_GlobalEnv


## 2.6 Exercises

>1. `lubridate::period()` returns an S4 class. What slots does it have? What class is each slot? What accessors does it provide?

In [20]:
wait <- lubridate::period(3, units = "days")
wait

[1] "3d 0H 0M 0S"

In [21]:
str(wait)

Formal class 'Period' [package "lubridate"] with 6 slots
  ..@ .Data : num 0
  ..@ year  : num 0
  ..@ month : num 0
  ..@ day   : num 3
  ..@ hour  : num 0
  ..@ minute: num 0


In [22]:
# slots
slotNames(wait)

[1] ".Data"  "year"   "month"  "day"    "hour"   "minute"

In [23]:
# class of each slot
getSlots(is(wait)) # getSlots("class name")

.Data      year     month       day      hour    minute 
"numeric" "numeric" "numeric" "numeric" "numeric" "numeric"

In [24]:
# accessors
sloop::s4_methods_class(is(wait))

Warning message in .S3methods(generic.function, class, envir):
"'class' is of length > 1; only the first element will be used"


generic,class,visible,source
<chr>,<chr>,<lgl>,<chr>
$,Period,TRUE,lubridate
$<-,Timespan,TRUE,lubridate
Arith,numeric,TRUE,lubridate
Arith,vector,TRUE,lubridate
Arith,Period,TRUE,lubridate
Arith,Timespan,TRUE,lubridate
Compare,numeric,TRUE,lubridate
Compare,vector,TRUE,lubridate
Compare,Period,TRUE,lubridate


In [25]:
wait$year
wait$day

[1] 0

[1] 3

# 3. Classes

## 3.1 Inheritance

## 3.2 Introspectation

>To determine what classes an object inherits from, use `is()`

In [26]:
is(new("Person"))
is(new("Employee"))

[1] "Person"

[1] "Employee" "Person"

>To test if an object inherits from a specific class, use the second argument of `is()`:

In [27]:
is(musk, "Person")

[1] TRUE

## 3.3 Redefinition

Read the book. What happens if you redefine a class that have existed?

## 3.4 Helper

A helper should always:

- Have the same name as the class, e.g. `myclass()`. 
- Have a thoughtfully crafted user interface with carefully chosen default values and useful conversions. 
- Create carefully crafted error messages tailored towards an end-user. 
- Finish by calling `methods::new()`.

In [28]:
# helper to create an instance of class Person
Person <- function(name, age = double()) {
    stopifnot(is.numeric(age))
    age <- as.double(age)
    # constructor
    new("Person", name = name, age = age)
}

In [29]:
conan <- Person("Edogawa Conan", 7L)
conan

An object of class "Person"
Slot "name":
[1] "Edogawa Conan"

Slot "age":
[1] 7


## 3.5 Validator

The constructor automatically checks that the slots have correct classes:

In [30]:
try(Person(mtcars))

Error in validObject(.Object) : 
  invalid class "Person" object: invalid object for slot "name" in class "Person": got class "data.frame", should be or extend class "character"


You will need to implement more complicated checks 

> Use **`setValidity`** to create a validator function for a class.  It takes a class and a function that returns TRUE if the input is valid, and otherwise returns a character vector describing the problem(s):

In [31]:
setValidity("Person", function(object) {
    # age must be non-negative
    if(object@age < 0)
        "@age must be a non-negative number"
    else TRUE
})

Class "Person" [in ".GlobalEnv"]

Slots:
                          
Name:       name       age
Class: character   numeric

Known Subclasses: "Employee"

You can no longer create an invalid object now

In [32]:
try(new("Person", name = "Trung", age = -2))

Error in validObject(.Object) : 
  invalid class "Person" object: @age must be a non-negative number


**NOTE**: The validity method is only called automatically by `new()`, so you can still create an invalid object by modifying it:

In [33]:
musk@age <- -1

>You can explicitly check the validity yourself by calling **`validObject()`**:

In [34]:
try(validObject(musk))

Error in validObject(musk) : 
  invalid class "Person" object: @age must be a non-negative number


<b style = "color:red">NOTE: You must always use `validObject` inside a getter method because we are changing the value of a slot</b>

In [ ]:
setMethod("age", "Person", function(object, value) {
    object@age <- value
    validObject(object)
    object
})

# 4 Generics and methods

Read the book

## 4.1 Signature

Read the book

## 4.2 Methods

Read the book

## 4.3 `show` method

Like `print` for S3 class. It controls how the object appears when it is printed. To define a method for an existing generic, you must first determine the arguments. You can get those from the documentation or by looking at the `args()` of the generic:

In [35]:
args(getGeneric("show"))

function (object) 
NULL

From above, we can see that our show method needs to have a single argument object:

In [41]:
# set method show for class Person
setMethod("show", "Person", function(object) {
    cat("Hello!\n",
        "I am ", object@name, "\n",
        "I am ", object@age, " years old")
})

In [42]:
musk

Hello!
 I am  Elon Musk 
 I am  -1  years old

## 4.4. Accessors, Getters and Setters

Read the book for in-depth explanation. Covered in Section 1.

# 5. Method Dispatch

**Must Read**, too long that I do not want to take notes. 
- Why method dispatch is used? Given a generic and a class, find the methods for that class
- What is method dispatch?
- How to perform sigle dispatch, multiple dispatch on single, multiple inheritance?



# 6. S4 and S3

## 6.1 How to make a slot in S4 to be S3 class?

 >To use an S3 class, you must first register it with **`setOldClass()`**

In [48]:
setOldClass(c("ordered","factor"))

However, it’s generally better to be more specific and provide a full S4 definition with slots and a prototype:

In [49]:
setClass("factor",
  contains = "integer",
  slots = c(
    levels = "character"
  ),
  prototype = structure(
    integer(),
    levels = character()
  )
)
setOldClass("factor", S4Class = "factor")

Warning message in rm(list = what, pos = classWhere):
"object '.__C__factor' not found"


## 6.2 S4 inherits from S3

If an S4 object inherits from an S3 class or a base type, it will have a special virtual slot called `.Data`. This contains the underlying base type or S3 object:

In [50]:
RangeNumeric <- setClass(
    "rangeNumeric",
    contains = "numeric",
    slots = c(min = "numeric", max = "numeric"),
    prototype = structure(numeric(), min = NA_real_, max = NA_real_)
)

In [53]:
rn <- RangeNumeric(1:10, min = 1, max = 10)
rn@min
rn@max

[1] 1

[1] 10